In [2]:
import os
import pandas as pd

# Specify the directory containing the files
directory = 'Data/majority_vote'
output_file = 'original_filenames.txt'

# List all files in the directory
files = sorted([f for f in os.listdir(directory) if f.endswith('.xlsx') and not f.startswith('~$')])

# Open a file to write the original and new filenames
with open(os.path.join(directory, output_file), 'w') as f:
    # Iterate through each file
    for i, file in enumerate(files):
        original_file_path = os.path.join(directory, file)
        
        # Define new file name
        new_file_name = f'conv{i+1}.xlsx'
        new_file_path = os.path.join(directory, new_file_name)
        
        # Rename the file
        os.rename(original_file_path, new_file_path)
        
        # Write the mapping of new file name to original file name
        f.write(f'{new_file_name} ---> {file}\n')
        
        # Read the Excel file
        df = pd.read_excel(new_file_path, engine='openpyxl', dtype=str)
        
        # Replace NaN values with 'NULL'
        df.fillna('NULL', inplace=True)
        
        # Change the ID column values
        df['ID'] = ['mess' + str(j+1) for j in range(len(df))]
        
        # Save the corrected dataframe back to the Excel file
        df.to_excel(new_file_path, index=False, engine='openpyxl')

print("All files have been processed and renamed. The original filenames are stored in original_filenames.txt.")

All files have been processed and renamed. The original filenames are stored in original_filenames.txt.


In [3]:
import os
import pandas as pd
import re
from datetime import datetime

# Specify the directory containing the files
directory = 'Data/majority_vote'

# List all files in the directory
files = os.listdir(directory)

# Function to check if a string is in a time format
def is_time_format(s):
    time_only_pattern = re.compile(r'^\d{2}:\d{2}:\d{2}$')
    date_time_pattern = re.compile(r'^\[\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\]$')
    return bool(time_only_pattern.match(s)) or bool(date_time_pattern.match(s))

# Function to extract time from various time formats, including AM/PM formats and brackets
def extract_time(s):
    # Remove any surrounding brackets
    s = re.sub(r'[\[\]]', '', s)
    
    # Convert AM/PM time to 24-hour format
    try:
        time_obj = datetime.strptime(s, '%I:%M:%S %p')
        return time_obj.strftime('%H:%M:%S')
    except ValueError:
        pass

    # Extract time from string
    time_pattern = re.compile(r'\d{2}:\d{2}:\d{2}')
    match = time_pattern.search(s)
    if match:
        return match.group(0)
    return s  # If no match, return the original string

# Function to normalize name based on the given rules
def normalize_name(s):
    # Remove angle brackets
    s = re.sub(r'[<>]', '', s)
    # Remove underscores or hyphens and take the first part
    s = re.split(r'[_-]', s)[0]
    # Remove @ symbol at the beginning
    s = re.sub(r'^@', '', s)
    # Extract the first word if it starts with @
    if '@' in s:
        s = re.split(r'@', s)[1]
    # Remove numeric characters
    s = re.sub(r'\d', '', s)
    return s

def correct_columns(file_path):
    try:
        # Read the Excel file, treating "NULL" as a string rather than NaN
        df = pd.read_excel(file_path, engine='openpyxl', dtype=str)
        df.fillna('NULL', inplace=True)  # Replace NaN values with 'NULL'
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return

    # Check if the columns are swapped
    if all(is_time_format(str(x)) for x in df['NAME']) and not all(is_time_format(str(x)) for x in df['TIME']):
        # Swap the values in NAME and TIME columns
        df[['NAME', 'TIME']] = df[['TIME', 'NAME']]

    # Normalize the TIME column
    df['TIME'] = df['TIME'].apply(extract_time)

    # Normalize the NAME column
    df['NAME'] = df['NAME'].apply(normalize_name)

    # Ensure columns are in the correct order
    correct_order = ['ID', 'NAME', 'TIME', 'TEXT', 'ROLE', 'HATE', 'TARGET', 'VERBAL_ABUSE', 'INTENTION', 'CONTEXT','SENTIMENT']
    df = df[correct_order]

    # Save the corrected dataframe back to the Excel file
    try:
        df.to_excel(file_path, index=False, engine='openpyxl')
    except Exception as e:
        print(f"Error saving {file_path}: {e}")

# Iterate through each file and correct the columns if necessary
for file in files:
    if file.endswith('.xlsx') and not file.startswith('~$'):  # Ensure we're only processing Excel files and ignore temporary files
        file_path = os.path.join(directory, file)
        correct_columns(file_path)

print("All files have been processed.")


All files have been processed.


In [4]:
import os
import pandas as pd

# Specify the directory containing the files
directory = 'Data/majority_vote'

# List all files in the directory
files = sorted([f for f in os.listdir(directory) if f.endswith('.xlsx') and not f.startswith('~$')])

# Process each file
for file in files:
    file_path = os.path.join(directory, file)
    
    # Read the Excel file
    df = pd.read_excel(file_path, engine='openpyxl', dtype=str)
    
    # Replace NaN values with 'NULL'
    df.fillna('NULL', inplace=True)
    
    # Change the ID column values
    df['ID'] = f'{file[:-5]}_mess' + (df.index + 1).astype(str)
    
    # Save the corrected dataframe back to the Excel file
    df.to_excel(file_path, index=False, engine='openpyxl')

print("ID column updated in all Excel files.")


ID column updated in all Excel files.


In [9]:
import os
import pandas as pd
import glob

def extract_annotation(directory, fields):
    extracted_data = []
    # Dynamically retrieve all .xlsx files in the specified directory
    filenames = glob.glob(os.path.join(directory, '*.xlsx'))
    
    if not filenames:
        print(f"No Excel files found in the directory {directory}.")
        return pd.DataFrame(columns=fields)

    for filepath in filenames:
        filename = os.path.basename(filepath)
        try:
            df = pd.read_excel(filepath)
            df.fillna('NULL', inplace=True)  # Replace NaN values with 'NULL'
            if set(fields).issubset(df.columns):
                extracted_data.append(df[fields])
            else:
                missing_fields = set(fields) - set(df.columns)
                print(f"File {filename} is missing fields: {missing_fields}")
        except Exception as e:
            print(f"Failed to process file {filename} due to error: {e}")

    if extracted_data:
        return pd.concat(extracted_data, ignore_index=True)
    else:
        print("No valid data extracted from the files.")
        return pd.DataFrame(columns=fields)



combined_data = extract_annotation(f'Data/majority_vote/', ['ID', 'NAME', 'TIME', 'TEXT', 'ROLE', 'HATE', 'TARGET', 'VERBAL_ABUSE', 'INTENTION', 'CONTEXT','SENTIMENT'])
if not combined_data.empty:
    
    output_path = f'Data/majority_vote/annotations_all.csv'
    combined_data.to_csv(output_path, index=False)

    print(f"Saved extracted data to {output_path}")
else:
    print(f"No data extracted for target: {target}")

Saved extracted data to Data/majority_vote/annotations_all.csv


In [15]:
import os
import pandas as pd
import random

# Function to extract unique names from a directory
def extract_unique_names(directory):
    unique_names = set()
    
    # Specify the filename explicitly (assuming it is a CSV file)
    filename = "annotations_all.csv"
    filepath = os.path.join(directory, filename)
    
    # Check if the file exists
    if os.path.exists(filepath):
        try:
            # Use pd.read_csv for CSV files
            df = pd.read_csv(filepath)
            
            # Check if 'NAME' column exists
            if 'NAME' in df.columns:
                unique_names.update(df['NAME'].dropna().unique())
        except Exception as e:
            print(f"Error reading file {filepath}: {e}")
    else:
        print(f"File {filename} does not exist in the directory {directory}.")
    
    return unique_names

# Extract unique names from the directory
unique_names = extract_unique_names("Data/majority_vote")
all_unique_names = list(unique_names)

# Generate 5-digit unique IDs for each name
name_to_id = {name: f'{random.randint(10000, 99999)}' for name in all_unique_names}

# Save the unique names and their IDs to a text file
unique_names_file = os.path.join("Data/majority_vote", 'unique_names.txt')
with open(unique_names_file, 'w') as file:
    for name, id_ in name_to_id.items():
        file.write(f'{name},{id_}\n')

print(f"Unique names and their IDs have been saved to {unique_names_file}")


Unique names and their IDs have been saved to Data/majority_vote/unique_names.txt


In [18]:
import os
import pandas as pd

# Function to transform the Hate field and add new fields
def LabelTransformation(directory):
    # Specify the file name
    filename = "annotations_all.csv"
    file_path = os.path.join(directory, filename)

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File {filename} does not exist in the directory {directory}.")
        return

    # Read the file as a CSV
    df = pd.read_csv(file_path)
    df.fillna('NULL', inplace=True)  # Replace NaN values with 'NULL'

    # Remove rows where 'ROLE' is 'conciliator'
    df = df[df['ROLE'] != 'conciliator']

    # Add the new "Hate_Abuse" field
    df['ABUSE'] = df['HATE'].apply(lambda x: 'YES' if x in ['OAG', 'CAG'] else 'NO')

    # Add the new "Behaviour Polarity" field
    df['B_POLARITY'] = df['INTENTION'].apply(lambda x: 'POS' if x in ['DFN', 'CNS', 'CR', 'EMP', 'OTH'] else 'NAG')

    # Add the new "Participant Role Detection" field
    df['POR'] = df['ROLE'].apply(
        lambda x: 'VS' if x in ['victim', 'victim_support'] else ('BS' if x in ['bully', 'bully_support'] else 'NULL')
    )

    # Save the updated DataFrame back to a CSV file
    df.to_csv(file_path, index=False)

# Call the function
LabelTransformation('Data/majority_vote')

In [22]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('Data/majority_vote/annotations_all.csv')

# Handle non-finite values by filling NaN with a default value (e.g., 0 or another placeholder)
# ABUSE
df['ABUSE'] = df['ABUSE'].replace({'YES': 1, 'NO': 0}).fillna(-1).astype(int)
    
# B_POLARITY
df['B_POLARITY'] = df['B_POLARITY'].replace({'POS': 0, 'NAG': 1}).fillna(-1).astype(int)

# POR
df['POR'] = df['POR'].replace({'BS': 1, 'VS': 0}).fillna(-1).astype(int)

# Save the updated DataFrame to a new CSV file
df.to_csv("Data/majority_vote/annotations_all_binary.csv", index=False)

print("File saved successfully with binary transformations!")

File saved successfully with binary transformations!


/tmp/ipykernel_21919/3841962762.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ABUSE'] = df['ABUSE'].replace({'YES': 1, 'NO': 0}).fillna(-1).astype(int)
/tmp/ipykernel_21919/3841962762.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['B_POLARITY'] = df['B_POLARITY'].replace({'POS': 0, 'NAG': 1}).fillna(-1).astype(int)
/tmp/ipykernel_21919/3841962762.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer